In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
from langchain_community.llms import HuggingFaceHub
import os
from google.colab import userdata

serp_key  = userdata.get('SERPAPI_KEY')
os.environ["SERPER_API_KEY"] = serp_key

In [ ]:
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=30879fb324a9ed6e0d3f3983296706df005f9e677bd4a3a2d3028164ddb7e3e4
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [7]:
#import os
#from utils import get_openai_api_key, get_serper_api_key
#from utils import get_serper_api_key


openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
# Agent 1: Factchecker
Factchecker = Agent(
    role="Ugandan News Factchecker",
    goal="Make sure to do amazing check on "
         "a fake news topic {topic} for a user",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a fact checker at PesaCheck, your prowess in "
        "navigating and extracting critical "
        "information from the internet."
        "Your skills help assess whether a piece of information like a question or statement in form of a topic {topic} "
        "is true or false. You use the google to check existing resources about "
        "the provided {topic} and if more than 3 newswebsites have reported about the {topic} "
        "you can confirm that the {topic} is true, inclusive when its less than 3 newswebsites "
        "have reported about the {topic} and false if no newswebsites have reported about "
        "the {topic} "
    )
)

In [9]:

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new fack check on the topic: {topic}. "
              "You base your writing on the work of "
              "the Factckcher, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Factckecher. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Fackchecker. "
              "You back up the information provided with sources "
              "remebering that where no sources exist then the topic: {topic} should be considered "
              "as false.",
    allow_delegation=False,
    verbose=True
)

In [10]:

editor = Agent(
    role="Editor",
    goal="Edit a given post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives content "
              "from the Content Writer. "
              "Your goal is to review the content "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [11]:
# Task for factcheker Agent
factcheck_task = Task(
    description=(
        "Perform a web search of the topic provided ({topic}) "
        "to check if there are any other websites talking about the topic: {topic}, if there are more then three "
        "different website, confirm the {topic} as true and say basing on these different sources. But if less than 3 sources exist "
        " just write that the {topic} is still inconclusive because not enough sources are talking about it and if zero sources are mentioning"
        " it then just consider the topic {topic} to be false."
    ),
    expected_output=(
        "A brief write up about the findings, including necessary "
        "sources if the findings are backed up by sources. if not then just a statement like False, no sources support the claim"
    ),
    agent=Factchecker,
    async_execution=True
)

In [12]:
# Task for Writer Agent

write = Task(
    description=(
        "1. Use the sources to craft a compelling "
            "paragraph post on {topic}.\n"
        "2. Start with the verdict on top.\n"
		    "3. Carefully explain in a paragraph why this was the "
            "verdict on the {topic}.\n"
        "4. You can list links to the sources if those that support the "
            "verdict exist and if none exist then this can be skipped, "
            "remember if the verdict is false then no need for sources.\n"
        "5. Proofread for grammatical errors and "
            "alignment with a journalistic's voice.\n"
    ),
    expected_output="A well-written writeup "
        "with a verdict on top, supported by a paragraph explaining the verdict below it, ready for publication, "
        "summarise the write up to only reveal relevant informtation",
    agent=writer,
)

In [13]:
# Task for Editor Agent

edit = Task(
    description=("Proofread the given write up for "
                 "grammatical errors and "
                 "alignment with a journalsitic voice."),
    expected_output="A well-written write up, "
                    "ready for publication, "
                    "in a summarised format.",
    context=[write, factcheck_task],
    agent=editor
)

#Other tasks are passed as context

In [14]:
crew = Crew(
    agents=[Factchecker,
            writer,
            editor],

    tasks=[factcheck_task,
           write,
           edit],

    verbose=True
)

In [15]:
### this execution will take a few minutes to run
result = crew.kickoff(inputs={"topic": "Is Museveni dead?"})

 [DEBUG]: == Working Agent: Ugandan News Factchecker
 [INFO]: == Starting Task: Perform a web search of the topic provided (Is Museveni dead?) to check if there are any other websites talking about the topic: Is Museveni dead?, if there are more then three different website, confirm the Is Museveni dead? as true and say basing on these different sources. But if less than 3 sources exist  just write that the Is Museveni dead? is still inconclusive because not enough sources are talking about it and if zero sources are mentioning it then just consider the topic Is Museveni dead? to be false.
 [DEBUG]: == [Ugandan News Factchecker] Task output: 


 [DEBUG]: == Working Agent: Content Writer
 [INFO]: == Starting Task: 1. Use the sources to craft a compelling paragraph post on Is Museveni dead?.
2. Start with the verdict on top.
3. Carefully explain in a paragraph why this was the verdict on the Is Museveni dead?.
4. You can list links to the sources if those that support the verdict exist a